# NOTE: no access to cluster yet, results are for sub 1

# Part I. Functional connectivity within subjects (standard FC)


<img src="http://drive.google.com/uc?export=view&id=1GPJ8XWcvqHr9NnyVm2DubagzV7cBPvsR" style="height:250px"/>

- The correlation of the time course of activity from the seed voxel with the time series from the target voxel is a proxy for the functional connectivity between those areas. 
- We have strong hypotheses about how information should flow in the brain. So instead of doing these analyses for the whole brain we do them for specific regions (`==ROI==pairs of voxels`).
- BOLD signal is noisy, remove noise and correct for movement $\rightarrow$ residuals. We do this analysis with the residuals, which is a proxy for the actual BOLD signal. 

> In other words, FC is a measure of the temporal correlation across different brain areas within a subjects brain.

See the [brainiak connectivity tutortial - 08](https://brainiak.org/tutorials/08-connectivity/) for more info.



<font color=red></font>

In [1]:
import warnings
import h5py
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
import numpy as np
import pandas as pd 

from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn import input_data

from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
from nibabel.affines import apply_affine
import nibabel as nib
import time

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio
from scipy import stats
from numpy.linalg import inv
from numpy import inf
from scipy import stats

# import own functions
import utils
reload(utils)

sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")

# 1. Load masks, residuals & specify params 

- Only take the residuals `'R'`, omit the rest of the variables.
- Save residuals as `.npy` files so that we can throw away the large files

In [2]:
path = '/Users/Daphne/data/'

# mask_nii is the functional mask, this selects the brain voxels
mask_nii = nib.load(os.path.join(path, 'mask.nii')) 
# this where we plot our mask ON (sometimes called brain_nii) - the anatomical/structural image
mean_nii = nib.load(os.path.join(path, 'mean.nii')) 

# inverse of the affine matrix: mni2cor
inv_affine = inv(mask_nii.affine) # get the transformation matrix

# load mask and get voxel coordinates
mask_arr = np.load(path+'mask_arr.npy') # all masks are the same
mask_mat = mask_arr[0] # so we can pick any one from the array
coords_mat = np.array(np.where(mask_mat == 1)) # so need one set of voxel coordinates for all
coords_mat[[0, 2]] = coords_mat[[2, 0]] # exchange the rows

# residuals
R = np.load(path+'residuals_sub1.npy')
R = np.swapaxes(R,1,0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Daphne/data/residuals_sub1.npy'

In [ ]:
R.shape # TR x voxels

In [ ]:
np.save('residuals_sub1', R)

In [ ]:
## uncomment and do for all subjects
# path = '/Users/Daphne/data/'
# filename = 'residuals_glm9_subj1_smooth.mat'

# data = h5py.File(path+filename,'r') 

# print(data.keys())
# print(data['R'].value.shape)

# residuals_sub1 = data['R'].value R# store residuals for subject

# np.save('residuals_sub1', residuals_sub1)

# 2. Pick a seed

- We loaded the whole-brain residuals
- Now, we pick ROIs (== seed voxel) and correlate their activity with other voxels in the brain
- If we find voxels that are correlated with a seed ROI, this suggests that these voxels are functionally connected

In [ ]:
# Theory ENCODING voxels
R_IFG_Tri_E = [42, 28, 26]
L_Insula_E = [-30, 28, 2]
R_DMPFC_E = [6, 38, 40]
L_IFG_Tri_E = [-50, 44, 12]
L_MTG_E = [-64, -50, 4]
R_MTG_E = [58, -36, 8]
Roi_1A = [48, 34,  8]

# Theory UPDATING voxels
R_IFG_Oper_U = [48, 12, 28]
L_PPC_U = [-56, -32, 46]
R_IFG_Tri_U = [52, 38, 16]
R_AG_U = [32, -60, 34]
L_Fusiform_U = [-40, -58, -12]
L_IFG_Oper_U = [-42, 4, 28]
R_PHC_U = [26, -42, -8]

# control voxels
# tip: can always try the contralateral ROIs: [-x y z]
L_lingual = [2, -86, 4]
Occipital = [-36 -88 -12]

In [ ]:
# create dictionary to map points to roi names
encoding_roi_dict = {'R_IFG_Tri_E':R_IFG_Tri_E, 'L_Insula_E':L_Insula_E, 'R_DMPFC_E':R_DMPFC_E,
                     'L_IFG_Tri_E':L_IFG_Tri_E, 'L_MTG_E':L_MTG_E, 'R_MTG_E':R_MTG_E, 'Roi_1A ':Roi_1A 
                    }

updating_roi_dict = {'R_IFG_Oper_U':R_IFG_Oper_U, 'L_PPC_U':L_PPC_U, 'R_IFG_Tri_U':R_IFG_Tri_U,
                     'R_AG_U':R_AG_U, 'L_Fusiform_U':L_Fusiform_U, 'L_IFG_Oper_U':L_IFG_Oper_U, 
                     'R_PHC_U':R_PHC_U
                    }

# combine in one
EU_dict = {**encoding_roi_dict, **updating_roi_dict}

# map control voxels to names
control_dict = {'control vox (left lingual)':L_lingual, 'control vox (Occipital)':Occipital}

In [ ]:
EU_dict

In [ ]:
# get the voxel indices for the theory encoding and theory updating regions
encoding_voxels = []
updating_voxels = []

# ENCODING ROIs
for key, value in encoding_roi_dict.items():

    coords_mni = value
    print(coords_mni)
    
    coords_natv = apply_affine(aff=inv_affine, pts=coords_mni) # from mni2cor
    vox_num = utils.get_vox_from_coords(coords_mat, coords_natv) # corresponding voxel
    
    encoding_voxels.append(vox_num)

# UPDATING ROIs
for key, value in updating_roi_dict.items():

    coords_mni = value
    print(coords_mni)
    
    coords_natv = apply_affine(aff=inv_affine, pts=coords_mni) # from mni2cor
    vox_num = utils.get_vox_from_coords(coords_mat, coords_natv) # corresponding voxel
    
    updating_voxels.append(vox_num)


In [ ]:
# === CHOOSE COORDINATES === (must be MNI)
coords_roi = R_IFG_Tri_E
coords_control = L_lingual
# ==========================

# Init the masking object
masker_ROI = input_data.NiftiSpheresMasker(
    coords_roi, 
    radius=8, standardize=True, t_r=2.,
    memory='nilearn_cache', memory_level=1, verbose=0
)

In [ ]:
def find_key(input_dict, value):
    return [k for k, v in input_dict.items() if v == value][0]

In [ ]:
#find_key(roi_dict, coords_roi)

## 2.2 Plot the signal for the ROI



In [ ]:
hdr = mask_nii.get_header()
hdr.get_xyzt_units() # so we see the voxel size & time units

In [ ]:
# get the corresponding voxel for ROI ...
native_coords_roi = apply_affine(aff=inv_affine, pts=coords_roi) # from mni2cor
roi_vox = utils.get_vox_from_coords(coords_mat, native_coords_roi) # corresponding voxel

# and control voxel
native_coords_control = apply_affine(aff=inv_affine, pts=coords_control) # from mni2cor
control_vox = utils.get_vox_from_coords(coords_mat, native_coords_control) # corresponding voxel

In [ ]:
"""
Plot the timeseries for the roi and control voxels
"""
voxel_ids = [roi_vox, control_vox]

plt.figure(figsize=(20, 5), dpi=100)
plt.title(f'Voxel activity for voxel ids = {voxel_ids}');
plt.plot(R[:, voxel_ids[0]], linewidth=1, label='ROI time series');
plt.plot(R[:, voxel_ids[1]], linewidth=1, label='Control time series');
plt.ylabel('Evoked activity');
plt.xlabel('Timepoints');
plt.legend();
sns.despine()

# 3. Compute the correlations with seed and control voxel

- In seed analysis we compute the cross correlation between the time series of the seed voxel and all other voxels. So we just do

```Python
For all voxels in whole brain mask

    correlate with seed voxel
```

### Caveats

- May have different hemodynamic lags in different regions



In [ ]:
def corr_with_seed(R, seed_vox):

    num_voxels = R.shape[1]
    seed_corr = np.zeros((num_voxels, 1)) # make volume to store the correlations
    
    for v in range(num_voxels):
        seed_corr[v, 0] = round(np.corrcoef(R[:,v], R[:, seed_vox])[0,1], 3) # take one value 

    # Transfrom the correlation values to Fisher z-scores    
    seed_corr_fishZ = np.arctanh(seed_corr)
    # arctanh(1)=inf so replace inf values with 2.7
    seed_corr_fishZ[seed_corr_fishZ == inf] = 2.7

    return seed_corr, seed_corr_fishZ

In [ ]:
def corr_with_control(R, control_vox):

    num_voxels = R.shape[1]
    control_corr = np.zeros((num_voxels, 1)) # make volume to store the correlations
    
    for v in range(num_voxels):
        control_corr[v, 0] = round(np.corrcoef(R[:,v], R[:, control_vox])[0,1],3) # take one value 

    # Transfrom the correlation values to Fisher z-scores    
    control_corr_fishZ = np.arctanh(control_corr)
    # arctanh(1)=inf so replace inf values with 2.7
    control_corr_fishZ[control_corr_fishZ == inf] = 2.7

    return control_corr, control_corr_fishZ

In [ ]:
# correlate all with seed voxel
corr_roi, corr_fz_roi = corr_with_seed(R=R, seed_vox=roi_vox)

# correlate all with control voxel
corr_control, corr_fz_control = corr_with_control(R=R, control_vox=control_vox)

In [ ]:
np.arctanh(0.99) # becomes inf

In [ ]:
corr_fz_roi.shape

In [ ]:
np.isinf(corr_fz_roi).any() # contains inf elements

In [ ]:
min(corr_fz_roi)

In [ ]:
np.sort(corr_fz_roi)

In [ ]:
print(corr_roi)

In [ ]:
plt.hist(corr_roi, bins=30)
plt.ylabel('Frequency');
plt.xlabel('r value');
plt.xlim([-1,1])
sns.despine()

In [ ]:
plt.hist(corr_control, bins=30)
plt.ylabel('Frequency');
plt.xlabel('r value');
plt.xlim([-1,1])
sns.despine()

In [ ]:
plt.hist(corr_fz_control, bins=30)
plt.ylabel('Frequency');
plt.xlabel('Fisher-z score');
sns.despine()

In [ ]:
plt.hist(corr_fz_roi, bins=30)
plt.ylabel('Frequency');
plt.xlabel('Fisher-z score');
sns.despine()

# 4. Visualise correlations on anatomical image

now we want to map the correlations back on the brain???

- Plot the seed correlation with every other voxel
- with stat_map
- convert correlations to nifti image?

[add_markers](https://github.com/nilearn/nilearn/blob/21fdc532dee895b5e46c3fc6a3d4f9803b247ac1/nilearn/plotting/displays.py#L923)

In [ ]:
coords = tuple(coords_mat) # needs to be a tuple in order to work

# 1) create a volume from mask_nii
volume = np.zeros(mask_nii.shape) 

# 2) Map the r values into brain space
volume[coords] = corr_fz_roi[:,0]

# 3) Create a nifti image from this with the affine from mask_nii
nii_corr_roi = nib.Nifti1Image(volume, mask_nii.affine, mask_nii.header)

In [ ]:
coords_roi

In [ ]:
# have to reshape in order to plot the seed on nifti object
coords_roi_arr = np.array(coords_roi)
coords_roi_arr = coords_roi_arr.reshape(1,3)
coords_roi_arr.shape

# One ROI

In [ ]:
f, ax = plt.subplots(1,1, figsize = (12, 5), dpi=70)
roi_name = find_key(roi_dict, coords_roi)

# Nilearn has useful tools for plotting our results as a map
r_map_ar = plotting.plot_stat_map(
    nii_corr_roi,
    threshold=0.6,
    cut_coords=coords_roi,
    annotate=True,
    bg_img=mean_nii,
    black_bg=False,
    axes=ax,
);

# Add the seed
r_map_ar.add_markers(
    marker_coords=coords_roi_arr, 
    marker_color='g',
    marker_size=40,
)

ax.set_title(f'Fisher z-score values for {roi_name}', size=15); 

In [ ]:
f, ax = plt.subplots(1,1, figsize = (12, 5), dpi=70)
roi_name = find_key(roi_dict, coords_roi)

# Create a glass brain
plotting.plot_glass_brain(
    nii_corr_roi,
    axes=ax,
    threshold=0.5,
    colorbar=True,  
    plot_abs=False,
    cut_coords=coords_roi,
    annotate=True,
    bg_img=mean_nii,
    black_bg=False,
    display_mode='lyrz', 
);

ax.set_title(f'Fisher z-score values for {roi_name}', size=15); 

# Display correlations with control value


In [ ]:
coords = tuple(coords_mat) # needs to be a tuple in order to work

# 1) create a volume from mask_nii
volume = np.zeros(mask_nii.shape) 

# 2) Map the r values into brain space
volume[coords] = corr_fz_control[:,0]

# 3) Create a nifti image from this with the affine from mask_nii
nii_corr_control = nib.Nifti1Image(volume, mask_nii.affine, mask_nii.header)

# have to reshape in order to plot the seed on nifti object
coords_control_arr = np.array(coords_control)
coords_control_arr = coords_control_arr.reshape(1,3)
coords_control_arr.shape

In [ ]:
f, ax = plt.subplots(1,1, figsize = (12, 5), dpi=70)
control_name = find_key(control_dict, coords_control)

# Nilearn has useful tools for plotting our results as a map
r_map_ar = plotting.plot_stat_map(
    nii_corr_control,
    threshold=0.6,
    cut_coords=coords_control,
    annotate=True,
    bg_img=mean_nii,
    black_bg=False,
    axes=ax,
);

# Add the seed
r_map_ar.add_markers(
    marker_coords=coords_control_arr, 
    marker_color='g',
    marker_size=40,
)

ax.set_title(f'Fisher z-score values for {control_name}', size=15); 

# 5. Correlation matrix and connectome

> Correlate the theory updating voxels with the theory encoding voxels

<img src="http://drive.google.com/uc?export=view&id=1RSGslepHFghu4LpvuwmRkcwdRCEYD4DP" style="height:200px"/>


In [ ]:
encoding_voxels

In [ ]:
# the voxels we need
updating_voxels

In [ ]:
# select the voxels from the residuals
updating_time_series = R[:, updating_voxels]
encoding_time_series = R[:, encoding_voxels]

N = len(encoding_time_series[1])
corr_matrix = np.zeros((N,N))


for i in range(N): # iterate through voxels (see img above)
    
    corr_col = []
    # pick the ith column from encoding 
    encoding_vec = encoding_time_series[:, i]
    
    # correlate it with each col (j) from updating time series
    for j in range(N):
        
        updating_vec = updating_time_series[:, j]
        
        # correlate column i with column j
        corr, _ = stats.pearsonr(encoding_vec, updating_vec)
    
        corr_col.append(round(corr,2)) 
    
    corr_matrix[:, i] = corr_col # insert correlations into correlation matrix
    print(f'--- Theory encoding col {i} ---')
    print(corr_matrix)
    

In [ ]:
f, ax = plt.subplots(1,1, figsize = (8, 7), dpi=70)

sns.heatmap(corr_matrix, 
            cmap='BuGn',
            xticklabels=encoding_roi_dict.keys(),
            yticklabels=updating_roi_dict.keys(),
            axes=ax,
            );

ax.set_xlabel('', fontsize=13);

# Create symmatric matrix just to make the connectome 

In [ ]:
all_roi_voxels = encoding_voxels + updating_voxels
all_roi_time_series = R[:, all_roi_voxels]

df = pd.DataFrame(all_roi_time_series) # convert to df
sym_corr_matrix = df.corr('pearson') # correlate all cols

f, ax = plt.subplots(1,1, figsize = (8, 7), dpi=70)

# Select upper triangle of correlation matrix
#upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
 
sns.heatmap(sym_corr_matrix,
            cmap='Greens',
            xticklabels=EU_dict.keys(),
            yticklabels=EU_dict.keys(),
            axes=ax
            );

In [ ]:
# insert null values for the parts where not interested in?? 

## TODO: add region labels & remove corrs we're not interested in

> Display only correlations > threshold

[plot_connectome](https://nilearn.github.io/modules/generated/nilearn.plotting.plot_connectome.html)

In [ ]:
f, ax = plt.subplots(1,1, figsize = (8, 7), dpi=100)

plotting.plot_connectome(adjacency_matrix=sym_corr_matrix,
                         node_coords=np.array(list(EU_dict.values())),
                         edge_threshold=0.45,
                         edge_cmap='Greens',
                         colorbar=True,
                         edge_vmax=1,
                         edge_vmin=0,
                         axes=ax
                         );

# Part II. Granger Causality

In [ ]:
all_roi_voxels = encoding_voxels + updating_voxels
all_roi_time_series = R[:, all_roi_voxels]

In [ ]:
all_roi_time_series.shape

In [ ]:
plt.figure(figsize=(20, 5), dpi=100)
plt.plot(all_roi_time_series[:,0])
plt.plot(all_roi_time_series[:,1])
plt.plot(all_roi_time_series[:,10]);

In [ ]:
plt.figure(figsize=(20, 5), dpi=100)
plt.plot(all_roi_time_series[:,0])
plt.plot(all_roi_time_series[:,7])

## TODO: implement this for all pairs of voxels from above and average over subjects

[statsmodels docs](https://www.statsmodels.org/stable/generated/statsmodels.tsa.stattools.grangercausalitytests.html)

The Null hypothesis for grangercausalitytests is that the time series in the second column, x2, does NOT Granger cause the time series in the first column, x1. Grange causality means that past values of x2 have a statistically significant effect on the current value of x1, taking past values of x1 into account as regressors. We reject the null hypothesis that x2 does not Granger cause x1 if the pvalues are below a desired size of the test.

The null hypothesis for all four test is that the coefficients corresponding to past values of the second time series are zero.

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

data = all_roi_time_series[:, [0,7]]

data.shape

In [ ]:
gc_res = grangercausalitytests(data, 4)